In [1]:
import gensim
import numpy as np
from gensim.models import KeyedVectors
import warnings
warnings.filterwarnings('ignore')

Load embeddings for ukrainian and russian.

In [41]:
uk_emb = KeyedVectors.load_word2vec_format("../cc.uk.300.vec")#, limit = 10000)

In [42]:
ru_emb = KeyedVectors.load_word2vec_format("../cc.ru.300.vec")#, limit = 10000)



Load small dictionaries for correspoinding words pairs as trainset and testset.

In [43]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930035829544067),
 ('апрель', 0.8729087710380554),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796871185303),
 ('февраль', 0.8401415944099426)]

In [44]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 1.0),
 ('липень', 0.9096440076828003),
 ('вересень', 0.9016969799995422),
 ('червень', 0.8992519974708557),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431818008423)]

In [45]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293080925941467),
 ('офор', 0.23256802558898926),
 ('deteydlya', 0.2303171455860138),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608885169029236),
 ('подход', 0.22305874526500702),
 ('иболее', 0.22003725171089172),
 ('2015Александр', 0.21872764825820923),
 ('конструктивен', 0.21796569228172302)]

In [8]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [11]:
uk_ru_train, X_train, Y_train = load_word_pairs("../ukr_rus.train.txt")

In [12]:
uk_ru_test, X_test, Y_test = load_word_pairs("../ukr_rus.test.txt")


Embedding space mapping

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:
$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$

or $$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit).

In [46]:
from sklearn.linear_model import LinearRegression
mapping = LinearRegression().fit(X_train, Y_train)
print(mapping.score(X_train, Y_train))
print(mapping.coef_)

0.7604608964938944
[[-0.13853334  0.03785874  0.00846561 ...  0.12043508  0.1230209
   0.05481485]
 [ 0.01697174  0.00580407  0.03639334 ...  0.02279952  0.11457596
  -0.00381462]
 [ 0.05413085 -0.05171295  0.04750827 ...  0.05530106  0.06657097
  -0.03942306]
 ...
 [ 0.01256713 -0.04073871 -0.04768448 ... -0.05323557  0.11357719
   0.01744952]
 [-0.05282985  0.0651894  -0.03701065 ...  0.18449809  0.13728851
  -0.12552929]
 [ 0.00325489 -0.0685897   0.11475997 ... -0.01013895  0.12247851
  -0.09955335]]




Let's take a look at neigbours of the vector of word "серпень" ("август" in Russian) after linear transform.


In [47]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('июнь', 0.857708215713501),
 ('июль', 0.8437438011169434),
 ('сентябрь', 0.8341312408447266),
 ('апрель', 0.8312028646469116),
 ('октябрь', 0.8284209370613098),
 ('ноябрь', 0.8258079290390015),
 ('март', 0.818211555480957),
 ('август', 0.8082067370414734),
 ('декабрь', 0.8062343597412109),
 ('февраль', 0.7984046339988708)]

In [81]:
ru_emb.most_similar(mapping.predict(uk_emb["кіт"].reshape(1, -1)))

[('кот', 0.45249590277671814),
 ('пес', 0.4417025148868561),
 ('пёс', 0.42835986614227295),
 ('волк', 0.42355042695999146),
 ('поросенок', 0.41129735112190247),
 ('заяц', 0.41038671135902405),
 ('рыжий', 0.4096396863460541),
 ('щенок', 0.40759021043777466),
 ('котик', 0.40626364946365356),
 ('волчонок', 0.4060942828655243)]



We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).


In [48]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE   
        mapped_vector = mapped_vectors[i]
        if ru in [x for (x,_) in ru_emb.most_similar([mapped_vector],topn=topn)]:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [49]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [50]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [51]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

In [52]:
print(precision_top1)
print(precision_top5)

0.5461538461538461
0.7307692307692307



Making it better (orthogonal Procrustean problem)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. We can restrict transform $W$ to be orthogonal. Then we will solve next problem:
$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components: $$X^TY=U\Sigma V^T\text{, singular value decompostion}$$ $$W^*=UV^T$$



In [116]:
def learn_transform(X, Y):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOU CODE HERE
    U, s, V = np.linalg.svd(np.matmul(X_train.T,Y_train))
    W = np.matmul(U,V)
    return W

In [117]:
W = learn_transform(X_train, Y_train)

In [119]:
print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.6153846153846154
0.7615384615384615



UK-RU Translator

Now we are ready to make simple word-based translator: for earch word in source language in shared embedding space we find the nearest in target language.


In [62]:
with open("../fairy_tale.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [63]:
uk_sentences

['лисичка - сестричка і вовк - панібрат',
 'як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !',
 'дай мені огня " .',
 'а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .',
 'прибігла на поле , аж там пасуть хлопці бичків .',
 'вона і каже їм : " ей , хлопці !',
 'проміняйте мені бичка - третячка за маковий пирожок " .',
 'тії согласились ; так вона їм говорить : " смотріть же , ви не їжте зараз сього пирожка , а тоді уже розломите , як я заведу бичка за могилку ; а то ви його ні за що не розломите " .',
 'бачите вже - лисичка таки собі була розумна , що хоть кого да обманить .',
 'тії хлопці так і зробили , а лисичка як зайшла за могилу , да зараз у ліс і повернула , 

In [120]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    # YOUR CODE HERE
    words = sentence.split(' ')
    translation = []
    for word in words:
        try:
            emb = uk_emb[word]
            translation.append(ru_emb.most_similar([np.matmul(emb, W)], topn=1)[0][0])           
        except:
            translation.append(word)
    return ' '.join(translation)

In [194]:
print(translate("добре сало з горілкою , особливо з перцем"))

хорошо сало со водкой , особенно со перцем


In [123]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка — девочка и волк — панібрат

src: як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !
dst: как была себе лисичка , из и пошла раз до однії бабы добувать огнь ; вошла во хату из и говорит : " хороший день тебе , бабушку !

src: дай мені огня " .
dst: дай мне огня " .

src: а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .
dst: а баба только что вытащила со печи пирожок со маком , сладкий , из и лежала , чтобы он прохолов ; а лисичка чтó и підгледала , из притом что баба приподнялась во печь , чтобы достать огня , то лисичка сейчас ухватила пирожок из и пустить со хаты , из , пробежать , весь мак со его виїла , а туд

src: де се ти набрала стільки риби ? "
dst: куда чтó мы набрала столько рыбы ? "

src: вона каже : " наловила , вовчику - братику ! "
dst: она говорит : " наловила , вовчику — маме ! "

src: а собі на думці : " подожди , і я зроблю з тобою таку штуку , як і ти зо мною " .
dst: а себе по мнении : " подожди , и мной сделаю со тобой такую штуку , как и мы За ней " .

src: - " як же ти ловила ? "
dst: — " как то мы ловила ? "

src: - " так , вовчику , уложила хвостик в ополонку , вожу тихенько да й кажу ; ловися , рибка , мала і велика !
dst: — " так , вовчику , уложила хвостик во прорубь , вожу тихонько из и говорю ; ловися , рыбка , имела и большая !

src: коли хочеш , то і ти піди , налови собі " .
dst: когда хочешь , то и мы пойди , налови себе " .

src: він побіг да зробив так , як казала лисичка .
dst: он побежал из сделал так , как говорила лисичка .

src: а лисичка стала за деревом да й дивиться ; коли у вовчика зовсім хвостик примерз , вона тоді побігла в село да й кричить : " іді

In [124]:
translate(".")

'.'

In [195]:
translate("1, 3")

'1, 1'

In [126]:
translate("кіт зловив мишу")

'котик поймал мышь'

# Part 2 : German and French langs

Load embeddings for French and German.

In [80]:
fr_emb = KeyedVectors.load_word2vec_format("../cc.fr.300.vec",binary=False, limit = 10000)
de_emb = KeyedVectors.load_word2vec_format("../cc.de.300.vec",binary=False, limit = 10000)

Load word pairs (french,german) from .txt

In [81]:
def load_word_pairs(filename):
    fr_de_pairs = []
    fr_vectors = []
    de_vectors = []
    
    with open(filename, "r") as inpf:
        for line in inpf:
            fr, de = line.rstrip().split("\t")
            if fr not in fr_emb or de not in de_emb:
                continue
            fr_de_pairs.append((fr, de))
            fr_vectors.append(fr_emb[fr])
            de_vectors.append(de_emb[de])
            
    return fr_de_pairs, np.array(fr_vectors), np.array(de_vectors)

In [82]:
fr_de_train, X_train, Y_train = load_word_pairs("../fr-de.0-5000.txt")#("../ukr_rus.train.txt")#
de_de_test, X_test, Y_test = load_word_pairs("../fr-de.5000-6500.txt")

Get two transform matrices M1 and M2 for first 300 pairs and next 300 pairs respectively 

In [83]:
def learn_transform(X, Y):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOU CODE HERE
    U, s, V = np.linalg.svd(np.matmul(X_train.T,Y_train))
    W = np.matmul(U,V)
    return W

In [84]:
M1 = learn_transform(X_train[:300], Y_train[:300])

In [85]:
M2 = learn_transform(X_train[300:601], Y_train[300:601])

Calculate precision for M1 and M2 on different sets of pairs

In [86]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE   
        mapped_vector = mapped_vectors[i]
        if ru in [x for (x,_) in de_emb.most_similar([mapped_vector],topn=topn)]:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [87]:
print("pair1 M1: {}".format(precision(fr_de_train[:300], np.matmul(X_train[:300], M1))))
print("pair1 M2: {}".format(precision(fr_de_train[:300], np.matmul(X_train[:300], M2))))
print("pair2 M1: {}".format(precision(fr_de_train[300:601], np.matmul(X_train[300:601], M1))))
print("pair2 M2: {}".format(precision(fr_de_train[300:601], np.matmul(X_train[300:601], M2))))

pair1 M1: 0.8133333333333334
pair1 M2: 0.8133333333333334
pair2 M1: 0.8803986710963455
pair2 M2: 0.8803986710963455


Calculte Frobenius norm for matrix $|M1-M2|$

In [88]:
M1_M2 = np.linalg.norm((M1-M2),ord='fro')
M1_M2

0.0

Calculte $\frac{|M1-M2|}{|M1|}$ ,where "$| |$" is  Frobenius norm

In [89]:
M1_M2/np.linalg.norm(M1,ord='fro')

0.0

Check matrices orthogonality

In [90]:
print(np.matmul(M1,M1.T))

[[ 1.0000001e+00 -3.6554411e-08 -1.5832484e-08 ...  6.1700121e-09
   1.4217221e-08 -5.1921234e-08]
 [-3.6554411e-08  1.0000005e+00  5.5413693e-08 ... -1.2805685e-09
  -2.4214387e-08  1.3038516e-08]
 [-1.5832484e-08  5.5413693e-08  9.9999988e-01 ... -5.8207661e-10
  -1.1263182e-08  1.8626451e-08]
 ...
 [ 6.1700121e-09 -1.2805685e-09 -5.8207661e-10 ...  9.9999964e-01
  -2.5131158e-08  6.8685040e-09]
 [ 1.4217221e-08 -2.4214387e-08 -1.1263182e-08 ... -2.5131158e-08
   9.9999994e-01  3.6423444e-08]
 [-5.1921234e-08  1.3038516e-08  1.8626451e-08 ...  6.8685040e-09
   3.6423444e-08  1.0000004e+00]]


M1 is orthogonal

In [91]:
print(np.matmul(M2,M2.T))

[[ 1.0000001e+00 -3.6554411e-08 -1.5832484e-08 ...  6.1700121e-09
   1.4217221e-08 -5.1921234e-08]
 [-3.6554411e-08  1.0000005e+00  5.5413693e-08 ... -1.2805685e-09
  -2.4214387e-08  1.3038516e-08]
 [-1.5832484e-08  5.5413693e-08  9.9999988e-01 ... -5.8207661e-10
  -1.1263182e-08  1.8626451e-08]
 ...
 [ 6.1700121e-09 -1.2805685e-09 -5.8207661e-10 ...  9.9999964e-01
  -2.5131158e-08  6.8685040e-09]
 [ 1.4217221e-08 -2.4214387e-08 -1.1263182e-08 ... -2.5131158e-08
   9.9999994e-01  3.6423444e-08]
 [-5.1921234e-08  1.3038516e-08  1.8626451e-08 ...  6.8685040e-09
   3.6423444e-08  1.0000004e+00]]


M2 is orthogonal

The same for pairs (German,French)

In [92]:
def load_word_pairs(filename):
    fr_de_pairs = []
    fr_vectors = []
    de_vectors = []
    
    with open(filename, "r") as inpf:
        for line in inpf:
            fr, de = line.rstrip().split("\t")
            if fr not in de_emb or de not in fr_emb:
                continue
            fr_de_pairs.append((fr, de))
            fr_vectors.append(de_emb[fr])
            de_vectors.append(fr_emb[de])
            
    return fr_de_pairs, np.array(fr_vectors), np.array(de_vectors)

Load word pairs (german,french) from .txt

In [93]:
de_fr_train, X_train, Y_train = load_word_pairs("../de-fr.0-5000.txt")#("../ukr_rus.train.txt")#
de_fr_test, X_test, Y_test = load_word_pairs("../de-fr.5000-6500.txt")

Get two transform matrices M1 and M2 for first 300 pairs and next 300 pairs respectively 

In [94]:
M1 = learn_transform(X_train[:300], Y_train[:300])
M2 = learn_transform(X_train[300:601], Y_train[300:601])

Calculate precision for M1 and M2 on different sets of pairs

In [95]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE   
        mapped_vector = mapped_vectors[i]
        if ru in [x for (x,_) in fr_emb.most_similar([mapped_vector],topn=topn)]:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [96]:
print("pair1 M1: {}".format(precision(de_fr_train[:300], np.matmul(X_train[:300], M1))))
print("pair1 M2: {}".format(precision(de_fr_train[:300], np.matmul(X_train[:300], M2))))
print("pair2 M1: {}".format(precision(de_fr_train[300:601], np.matmul(X_train[300:601], M1))))
print("pair2 M2: {}".format(precision(de_fr_train[300:601], np.matmul(X_train[300:601], M2))))

pair1 M1: 0.7533333333333333
pair1 M2: 0.7533333333333333
pair2 M1: 0.7940199335548173
pair2 M2: 0.7940199335548173


Calculte Frobenius norm for matrix $|M1-M2|$

In [98]:
M1_M2 = np.linalg.norm((M1-M2),ord='fro')
M1_M2

0.0

Calculte $\frac{|M1-M2|}{|M1|}$ ,where "$| |$" is  Frobenius norm

In [99]:
M1_M2/np.linalg.norm(M1,ord='fro')

0.0

Check matrices orthogonality

In [100]:
print(np.matmul(M1,M1.T))

[[ 9.9999976e-01 -3.6321580e-08  4.8428774e-08 ...  4.8894435e-09
   3.5157427e-08 -5.4482371e-08]
 [-3.6321580e-08  1.0000000e+00 -9.3132257e-10 ... -1.7462298e-08
  -4.2608008e-08 -1.0186341e-08]
 [ 4.8428774e-08 -9.3132257e-10  9.9999970e-01 ...  4.5169145e-08
  -3.9814040e-08  6.9790985e-08]
 ...
 [ 4.8894435e-09 -1.7462298e-08  4.5169145e-08 ...  1.0000000e+00
   1.2805685e-09 -3.1956006e-08]
 [ 3.5157427e-08 -4.2608008e-08 -3.9814040e-08 ...  1.2805685e-09
   1.0000000e+00  1.8917490e-09]
 [-5.4482371e-08 -1.0186341e-08  6.9790985e-08 ... -3.1956006e-08
   1.8917490e-09  1.0000004e+00]]


M1 is orthogonal

In [101]:
print(np.matmul(M2,M2.T))

[[ 9.9999976e-01 -3.6321580e-08  4.8428774e-08 ...  4.8894435e-09
   3.5157427e-08 -5.4482371e-08]
 [-3.6321580e-08  1.0000000e+00 -9.3132257e-10 ... -1.7462298e-08
  -4.2608008e-08 -1.0186341e-08]
 [ 4.8428774e-08 -9.3132257e-10  9.9999970e-01 ...  4.5169145e-08
  -3.9814040e-08  6.9790985e-08]
 ...
 [ 4.8894435e-09 -1.7462298e-08  4.5169145e-08 ...  1.0000000e+00
   1.2805685e-09 -3.1956006e-08]
 [ 3.5157427e-08 -4.2608008e-08 -3.9814040e-08 ...  1.2805685e-09
   1.0000000e+00  1.8917490e-09]
 [-5.4482371e-08 -1.0186341e-08  6.9790985e-08 ... -3.1956006e-08
   1.8917490e-09  1.0000004e+00]]


M2 is orthogonal